In [9]:
import pandas as pd
import anndata as ad
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import anndata2ri
import rpy2.rinterface_lib.callbacks
import logging
import numpy as np
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

In [10]:
anndata2ri.activate()

/tmp/ipykernel_3136231/4016445965.py:1: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [11]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [12]:
adata = sc.read_h5ad("/data/hanxue/hanxue_data/adata-xwt-annotated_T-0429.h5ad")
adata

AnnData object with n_obs × n_vars = 48293 × 41733
    obs: 'batch', 'mt_frac', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'n_genes', 'n_counts', 'leiden', 'major_type', 'molecular_type', 'leiden_harmony', 'major_type_harmony', 'minor_type', 'minor-type', 'gender', 'doublet_score', 'predicted_doublet', 'leiden-harmony-female', 'majorType-Female', 'epi-subtype', 'treat', 'leiden-1', 'annotation', 'leiden-T', 'groups', 'anno-T', 'anno-subT'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'anno-T_colors', 'anno-subT_colors', 'annotation_colors', 'batch_colors', 'dendrogram_anno-T', 'dendrogram_annotation', 'dendrogram_leiden', 'dendrogram_leiden-1', 'gender_colors', 'hvg', 'leiden', 'leiden-1_colors', 'leiden-T_colors', 'leiden-harmony-female_colors', 'leiden_colors', 'leiden_harmony_co

In [13]:
# adata.obs['batch'].unique()

In [14]:
adata = adata[~adata.obs['batch'].isin(['16T'])].copy()
adata.obs['batch'].unique()

['1T', '2T', '8T', '9T', '10T', ..., '17T', '19T', '20T', '21T', '22T']
Length: 17
Categories (17, object): ['1T', '2T', '8T', '9T', ..., '19T', '20T', '21T', '22T']

In [15]:
# 标注数据
def anno(adata:ad.AnnData, annoDict:dict, obsKey='cnv_status', obsVal='cnv_leiden', default="Unknown"):
  if default is not None:
    adata.obs[obsKey] = default
  for key in annoDict.keys():
    adata.obs.loc[adata.obs[obsVal].isin(annoDict[key]), obsKey] = key
  return adata

# python环境中准备miloR数据（需要跑过sc.tl.neighbors）
def prepare_miloR(adata: ad.AnnData, conDict: dict, condName, condVal, batch='batch'):
    adata_no_knn = adata.copy()
    adata_no_knn.obsp = None
    adata_no_knn.uns.pop("neighbors")
    
    # 标注分组
    adata_no_knn = anno(adata_no_knn, annoDict=conDict, obsKey=condName, obsVal=condVal, default=None)
    adata_no_knn.obs[condName] = adata_no_knn.obs[condName].astype('category')
    used_cond = adata_no_knn.obs[condName].cat.categories
    adata_no_knn = adata_no_knn[adata_no_knn.obs[condName].isin(used_cond)]  # 仅保留指定分组
    
    print(adata_no_knn.obs[condName].cat.categories)
    
    # 邻接矩阵
    knn_adjacency = adata.obsp["connectivities"]
    
    # 构造 design_df：两列，无索引
    design_df = adata_no_knn.obs[[batch, condName]].drop_duplicates().reset_index(drop=True)
    
    return adata_no_knn, knn_adjacency, design_df

#以标注TNBC数据为例
conDict = {
  'Post': ['After'],
  'Others': ['Before', 'W/O Treat']
}
adata_no_knn, knn_adjacency, design_df = prepare_miloR(adata, conDict, 'TorN', 'treat') # 最后为分组依据

Index(['Others', 'Post'], dtype='object')


In [16]:
adata_no_knn.write_h5ad("adata-for-milo.h5ad")

In [17]:
from scipy.io import mmwrite
from scipy.sparse import csr_matrix

# 假设 knn_adjacency 是 csr_matrix，如果不是请转换
knn_adjacency = knn_adjacency.tocsr() # 确保格式兼容
mmwrite("knn_adjacency.mtx", knn_adjacency)

In [18]:
design_df.to_csv("design_matrix.csv", index=False)